#### 다음 실습 코드는 학습 목적으로만 사용 바랍니다. 문의 : audit@korea.ac.kr 임성열 Ph.D.

Groq의 빠른 추론 엔진을 LangChain에서 체험하는 코드를 소개합니다.    
langchain_groq를 통해 쉽게 연결할 수 있습니다.

In [1]:
# Agent처럼 별도 다른 버전으로 맞춘 패키지를 별도 가상 환경에서 동작시키기 위한 예시 (다음 코드는 최신 버전에서도 정상 동작함)
# 일반적으로 최신 버전으로 지정해야 정상 처리되나, 특정 헤비한 패키지인 경우 하위 버전 종속성 가짐 (예, Crew AI, TensorFlow, PyTorch 등) 
!pip install langchain_groq pymupdf pypdf beautifulsoup4 requests \
            langchain==0.1.14 \
            langchain_community==0.0.30 \
            openai==1.25.1

  Using cached langchain_groq-0.3.7-py3-none-any.whl (16 kB)
  Using cached pymupdf-1.26.3-cp39-abi3-win_amd64.whl (18.7 MB)
  Using cached pypdf-6.0.0-py3-none-any.whl (310 kB)
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
  Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Using cached langchain-0.1.14-py3-none-any.whl (812 kB)
  Using cached langchain_community-0.0.30-py3-none-any.whl (1.9 MB)
  Using cached openai-1.25.1-py3-none-any.whl (312 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl (161 kB)
  Using cached sqlalchemy-2.0.43-cp311-cp311-win_amd64.whl (2.1 MB)
  Using cached aiohttp-3.12.15-cp311-cp311-win_amd64.whl (453 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl (303 kB)
  Using cached langchain_text_splitters-0.0.2-py3-none-any.whl (23 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl (311 kB)
  Using ca


[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


현재 사용 가능한 주요 모델과 API 제한은 아래와 같습니다.


| ID                                     | Requests per Minute | Requests per Day | Tokens per Minute | Tokens per Day |
|----------------------------------------|---------------------|------------------|-------------------|----------------|
| gemma-7b-it                            | 30                  | 14,400           | 15,000            | 500,000        |
| gemma2-9b-it                           | 30                  | 14,400           | 15,000            | 500,000        |
| llama-3.1-70b-versatile                | 30                  | 14,400           | 18,000            | 500,000        |
| llama-3.1-8b-instant                   | 30                  | 14,400           | 20,000            | 500,000        |
| llama-3.2-11b-text-preview             | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-11b-vision-preview           | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-1b-preview                   | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-3b-preview                   | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-90b-text-preview             | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-90b-vision-preview           | 15                  | 3,500            | 7,000             | 250,000        |
| llava-v1.5-7b-4096-preview             | 30                  | 14,400           | 30,000            | (No limit)     |
| mixtral-8x7b-32768                     | 30                  | 14,400           | 5,000             | 500,000        |


In [6]:
from langchain_groq import ChatGroq # Groq-LangChain 연결

from langchain_community.document_loaders import WebBaseLoader

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import time

간단한 체인을 만들고 실행합니다.

https://console.groq.com/keys 에서 키를 생성해야 합니다.

In [7]:
import os
os.environ['GROQ_API_KEY'] = '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' #기존 groq Key로 여러명이 동시에 동작시키면 차단될 수 있습니다. https://groq.com/ DEV CONSOLE에서 접속하여 여러분의 개별 groq Key를 넣어주세요.

import time
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 환경변수로 Groq API Key 설정
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY", "여기에_본인_KEY를_입력하세요")

# Groq Chat 모델 설정 (텍스트 전용 작업에는 llama3-8b-8192 등 사용 권장)
chat = ChatGroq(
    temperature=0.1,
    model="llama3-8b-8192",  # 또는 "llama3-70b-8192"
)

# 웹 페이지 로드
url = "https://wow.groq.com/why-groq/"
loader = WebBaseLoader(url)
docs = loader.load()

# 예외 처리: 문서가 없을 경우 중단
if not docs:
    raise ValueError("❗ 문서 로드 실패: 페이지에서 내용을 불러올 수 없습니다.")

# 질문 및 프롬프트 구성
question = "Groq 엔진의 LLM 추론 속도가 빠른 이유는 무엇인가요? 한국어로 답변하세요."

system = "Answer the question from given contexts. Answer in Korean."
human = """
Context: {context}

---

Question: {question}
"""

# 체인 구성
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
chain = prompt | chat | StrOutputParser()

# 실행 및 출력
start = time.time()
result = chain.invoke({
    "context": docs[0].page_content,
    "question": question
})
print("🔹 답변:\n", result)

end = time.time()
print(f"\n⏱️ 처리 시간: {end - start:.2f}초")


🔹 답변:
 Groq 엔진의 LLM 추론 속도가 빠른 이유는 다음과 같습니다.

Groq 엔진은 특화된 하드웨어 아키텍처를 사용하여 LLM(Large Language Model) 추론을 최적화합니다. 이를 통해 Groq 엔진은 일반적인 CPU보다 훨씬 빠른 추론 속도를 달성할 수 있습니다. 또한, Groq 엔진은 저용량의 메모리와 고성능의 처리 능력을 결합하여 LLM 추론을 더 빠르게 수행할 수 있습니다.

따라서, Groq 엔진의 LLM 추론 속도가 빠른 것은 특화된 하드웨어 아키텍처와 저용량의 메모리, 고성능의 처리 능력 등 다양한 요인 때문입니다.

⏱️ 처리 시간: 6.58초


## Summarization

요약은 LLM의 아주 중요한 기능 중 하나입니다.   
일반적으로, LLM의 Abstractive Summarization은 3개의 방법을 사용합니다.

- Stuff : 전체 코퍼스를 하나의 프롬프트에 넣고 요약 생성하기
- Map-Reduce : 코퍼스를 청크로 분리하고, 각 청크의 요약을 생성한 뒤 합치기
- Refine: 코퍼스를 청크로 분리하고, 순차적으로 읽으며 요약 업데이트하기



## Stuff

gemma 2 모델을 이용해 요약을 수행해 보겠습니다.

In [8]:
# Stuff
example_URL='https://arxiv.org/abs/1706.03762'

loader = WebBaseLoader(example_URL)
docs = loader.load()


gemma2 = ChatGroq(
    temperature=0,
    model="gemma2-9b-it",
)

summarize_prompt = ChatPromptTemplate.from_messages([
    ('system', '''Summarize the following paper in Korean.
Emphasize the uniqueness and contribution of the paper.
Answer should be in 10 sentences.
Please Answer in Korean.
'''),
    ('user', '''{text}''')])

print(len(docs[0].page_content))
summarize_prompt.format_messages(text=docs[0].page_content)

5951


[SystemMessage(content='Summarize the following paper in Korean.\nEmphasize the uniqueness and contribution of the paper.\nAnswer should be in 10 sentences.\nPlease Answer in Korean.\n'),
 HumanMessage(content="\n\n [1706.03762] Attention Is All You Need\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n\nWe gratefully acknowledge support from the Simons Foundation, member institutions, and all contributors.\nDonate\n\n\n\n\n\n > cs > arXiv:1706.03762\n  \n\n\n\n\n\nHelp | Advanced Search\n\n\n\n\nAll fields\nTitle\nAuthor\nAbstract\nComments\nJournal reference\nACM classification\nMSC classification\nReport number\narXiv identifier\nDOI\nORCID\narXiv author ID\nHelp pages\nFull text\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nopen search\n\n\n\n\n\n\nGO\n\n\n\nopen navigation menu\n\n\nquick links\n\nLogin\nHelp Pages\nAbout\n\n\n\n\n\n\n\n\n\n\n\n\nComputer Science > Computation and Language\n\n\narXiv:1706.03762 (cs)\n 

In [9]:
summarize_chain = summarize_prompt | gemma2 | StrOutputParser()
summary = summarize_chain.invoke(docs[0].page_content)
print(summary)

이 논문 "Attention Is All You Need"는 순환 신경망이나 합성곱 신경망과 같은 복잡한 구조를 사용하지 않고, **단순히 Attention 메커니즘만을 기반으로** 기존의 기계 번역 모델을 뛰어넘는 성능을 보여주는 새로운 네트워크 아키텍처인 Transformer를 제안합니다. 

이 논문의 가장 큰 혁신은 **RNN이나 CNN을 사용하지 않고도 효과적인 기계 번역 모델을 구축할 수 있다는 것을 증명**한 것입니다. 

Transformer는 Attention 메커니즘을 통해 입력 시퀀스의 모든 요소 간의 관계를 효과적으로 학습할 수 있기 때문에, **더 빠른 학습 속도와 높은 병렬 처리 능력**을 제공합니다. 

실험 결과, Transformer는 기존 최고 성능 모델을 뛰어넘는 결과를 보였으며, 특히 **훈련 시간이 훨씬 단축**되는 것을 확인할 수 있습니다. 

또한, Transformer는 **기계 번역 외에도 문장 구조 분석과 같은 다른 NLP 작업에도 효과적으로 적용**될 수 있음을 보여주었습니다. 

이 논문은 **Attention 메커니즘의 잠재력을 보여주는 중요한 연구**이며, 이후 다양한 NLP 모델에 큰 영향을 미쳤습니다. 

결론적으로, Transformer는 **기계 학습 분야에 혁신적인 변화를 가져온** 중요한 논문입니다. 





해당 방법은 매우 간단하지만, Context 길이를 넘어서는 경우 에러가 발생합니다.

## Map-Reduce
LangChain의 PyMuPdfLoader를 이용하여 임의의 PDF를 요약해 보겠습니다.

In [7]:
# # Password 있는 PDF 열기
# from langchain.document_loaders import PyPDFLoader
# path_material = './paper-attention.pdf'

# pypdf_loader = PyPDFLoader(path_material, password='비밀번호')
# material_pages = pypdf_loader.load()

In [10]:
from langchain_community.document_loaders import PyMuPDFLoader
import urllib.request

paper_URL = "https://arxiv.org/pdf/1706.03762"


# 외부 링크에서 PDF 파일을 다운로드하는 코드
urllib.request.urlretrieve(
    paper_URL,
    filename="paper-attention.pdf"
)
path = './paper-attention.pdf'

In [11]:
from langchain_core.documents import Document


loader = PyMuPDFLoader(path)
# 페이지별로 저장
pages = loader.load()

# 코퍼스에 모두 결합
corpus = Document(page_content='')
for page in pages:
    corpus.page_content += page.page_content
len(corpus.page_content)

39498

긴 Context를 처리하기 위해, 전체 코퍼스를 작은 단위로 쪼개 보겠습니다.

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000) #chunk_size는 한 chunkdp 포함될 최대 글자수입니다.
document_list = text_splitter.split_documents([corpus])
len(document_list)

5

이후 Map-Reduce와 Refine을 사용할 수 있습니다.

## Map-Reduce

In [13]:
from tqdm import tqdm

# Map 과정 : 각 문서에 대해 요약을 생성합니다. -> Long Context 인 경우, 균일하게 분할하여 요약하고 다시 합치는 방법으로 성능 개선

map_prompt = ChatPromptTemplate.from_messages([
    ('system', '''주어진 논문의 내용을 읽고 한국어로 요약하세요.
요약은 1개의 문단과 문단별 6개의 문장으로 작성하세요.
답변은 한국어로 작성하세요.
같은 내용을 반복하지 마세요.
'''),
    ('user', '''{text}''')])

map_chain  = map_prompt | gemma2 | StrOutputParser()

raw_summaries = []
for i in tqdm(range(len(document_list))):
    response = map_chain.invoke(document_list[i].page_content)
    raw_summaries.append(response)
    print('')
    print('#',i)
    print(response)
    print('===========================')


 20%|██        | 1/5 [00:06<00:24,  6.21s/it]


# 0
본 논문은 'Transformer'라는 새로운 신경망 아키텍처를 소개하며, 이 아키텍처는 주어진 문장의 모든 단어들 간의 관계를 파악하기 위해 주의 메커니즘만을 사용하여 기존의 순환 신경망이나 합성곱 신경망을 사용하지 않습니다. 

Transformer는 인코더와 디코더로 구성되며, 각각은 여러 개의 셀프-어텐션 계층과 포인트 와이즈 풀리 연결 계층으로 이루어져 있습니다. 인코더는 입력 문장을 벡터 형태로 변환하고, 디코더는 이 벡터를 기반으로 출력 문장을 생성합니다. 

Transformer는 셀프-어텐션을 통해 입력 문장의 모든 단어들 간의 관계를 효과적으로 학습할 수 있습니다. 이는 기존의 순환 신경망이나 합성곱 신경망에 비해 병렬 처리가 가능하며, 훈련 시간을 크게 단축시킬 수 있습니다. 

실험 결과, Transformer는 기존의 최고 성능 모델을 능가하는 결과를 보였으며, 특히 기존 모델보다 훨씬 짧은 시간 안에 훈련될 수 있다는 장점을 보여주었습니다. 

결론적으로, Transformer는 뛰어난 성능과 효율성을 제공하는 새로운 신경망 아키텍처이며, 향후 기계 번역 및 기타 자연어 처리 분야에서 폭넓게 활용될 것으로 기대됩니다. 





 40%|████      | 2/5 [00:27<00:45, 15.09s/it]


# 1
본 논문은 Transformer 모델의 핵심 구성 요소인 Self-Attention 메커니즘을 설명하고, 이를 기반으로 구축된 모델의 장점을 다룹니다. 

먼저, Scaled Dot-Product Attention 알고리즘을 소개하며, 이는 쿼리, 키, 값을 이용하여 각 단어의 중요도를 계산하는 방식을 설명합니다. 이 알고리즘은 dot product를 이용하여 계산 속도와 공간 효율성을 높였으며, softmax 함수를 통해 각 단어의 가중치를 결정합니다. 

다음으로, Multi-Head Attention을 통해 여러 관점에서 정보를 동시에 처리할 수 있도록 개선된다고 설명합니다. 각 헤드는 독립적으로 작동하며, 여러 헤드의 결과를 결합하여 더 풍부한 정보를 학습할 수 있도록 합니다. 

논문에서는 Transformer 모델에서 Self-Attention이 사용되는 세 가지 방식을 제시합니다. 첫째, encoder-decoder attention은 디코더의 각 단어가 인코더의 모든 단어에 대해 주의를 기울일 수 있도록 합니다. 둘째, encoder 내부의 Self-Attention은 각 단어가 이전 단어들과의 관계를 학습할 수 있도록 합니다. 셋째, 디코더 내부의 Self-Attention은 각 단어가 이전 단어들과의 관계를 학습하면서도 오른쪽으로만 정보를 흐르도록 제한합니다. 

또한, Positional Encoding을 통해 순서 정보를 모델에 제공한다고 설명합니다. 

마지막으로, Self-Attention이 Recurrent와 Convolutional layer보다 뛰어난 성능을 보이는 이유를 설명합니다. Self-Attention은 모든 단어 간의 관계를 효율적으로 학습할 수 있으며, Long-range dependency를 학습하는 데 유리합니다.





 40%|████      | 2/5 [01:35<02:22, 47.57s/it]


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `gemma2-9b-it` in organization `org_01jd3jew9veemay6ep0tb2znnz` service tier `on_demand` on tokens per minute (TPM): Limit 15000, Used 12864, Requested 2574. Please try again in 1.751s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [14]:
raw_summaries

["본 논문은 'Transformer'라는 새로운 신경망 아키텍처를 소개하며, 이 아키텍처는 주어진 문장의 모든 단어들 간의 관계를 파악하기 위해 주의 메커니즘만을 사용하여 기존의 순환 신경망이나 합성곱 신경망을 사용하지 않습니다. \n\nTransformer는 인코더와 디코더로 구성되며, 각각은 여러 개의 셀프-어텐션 계층과 포인트 와이즈 풀리 연결 계층으로 이루어져 있습니다. 인코더는 입력 문장을 벡터 형태로 변환하고, 디코더는 이 벡터를 기반으로 출력 문장을 생성합니다. \n\nTransformer는 셀프-어텐션을 통해 입력 문장의 모든 단어들 간의 관계를 효과적으로 학습할 수 있습니다. 이는 기존의 순환 신경망이나 합성곱 신경망에 비해 병렬 처리가 가능하며, 훈련 시간을 크게 단축시킬 수 있습니다. \n\n실험 결과, Transformer는 기존의 최고 성능 모델을 능가하는 결과를 보였으며, 특히 기존 모델보다 훨씬 짧은 시간 안에 훈련될 수 있다는 장점을 보여주었습니다. \n\n결론적으로, Transformer는 뛰어난 성능과 효율성을 제공하는 새로운 신경망 아키텍처이며, 향후 기계 번역 및 기타 자연어 처리 분야에서 폭넓게 활용될 것으로 기대됩니다. \n\n\n",
 '본 논문은 Transformer 모델의 핵심 구성 요소인 Self-Attention 메커니즘을 설명하고, 이를 기반으로 구축된 모델의 장점을 다룹니다. \n\n먼저, Scaled Dot-Product Attention 알고리즘을 소개하며, 이는 쿼리, 키, 값을 이용하여 각 단어의 중요도를 계산하는 방식을 설명합니다. 이 알고리즘은 dot product를 이용하여 계산 속도와 공간 효율성을 높였으며, softmax 함수를 통해 각 단어의 가중치를 결정합니다. \n\n다음으로, Multi-Head Attention을 통해 여러 관점에서 정보를 동시에 처리할 수 있도록 개선된다고 설명합니다. 각 헤드는 독립적으로 작동하며, 여러 헤드의 결과를 결합하여 더 풍부한 정보를 학습할 수 있도록 

In [15]:
# Reduce 과정 : 각 문서의 요약을 하나로 합칩니다.
reduce_prompt = ChatPromptTemplate.from_messages([

    ('system', '''
Generate a summary of the following text that includes the following elements:

* A title that accurately reflects the content of the text.
* An introduction paragraph that provides an overview of the topic.
* Bullet points that list the key points of the text.
* A conclusion paragraph that summarizes the main points of the text.

Answer in Korean.
'''),
    ('user', '''{text}
---
요약(In Korean):
''')])

reduce_chain = reduce_prompt | gemma2 | StrOutputParser()

summary = reduce_chain.invoke('\n---\n'.join(raw_summaries))

print(summary)


## Transformer: 자연어 처리 분야의 혁신적인 신경망 아키텍처

본 논문은 자연어 처리 분야에서 혁신적인 성과를 보여주는 새로운 신경망 아키텍처인 Transformer를 소개합니다. Transformer는 기존의 순환 신경망이나 합성곱 신경망을 대체하여, 주어진 문장의 모든 단어들 간의 관계를 파악하는 데 탁월한 성능을 보이는 Self-Attention 메커니즘을 사용합니다. 

* Transformer는 인코더와 디코더로 구성되며, 각각은 셀프-어텐션 계층과 포인트 와이즈 풀리 연결 계층으로 이루어져 있습니다.
* 셀프-어텐션을 통해 Transformer는 입력 문장의 모든 단어들 간의 관계를 효과적으로 학습할 수 있습니다. 
* 이는 병렬 처리가 가능하여 훈련 시간을 크게 단축시키는 장점을 제공합니다.
* 실험 결과, Transformer는 기존 최고 성능 모델을 능가하는 성능을 보였으며, 짧은 시간 안에 훈련될 수 있다는 장점을 보여주었습니다.
* Transformer는 Scaled Dot-Product Attention, Multi-Head Attention, Positional Encoding 등의 핵심 구성 요소를 통해 자연어의 복잡한 구조를 효과적으로 학습합니다.

결론적으로, Transformer는 뛰어난 성능과 효율성을 제공하는 새로운 신경망 아키텍처로, 기계 번역, 텍스트 요약, 질의응답 등 다양한 자연어 처리 분야에서 폭넓게 활용될 것으로 기대됩니다. 





## Refine

Refine은 청크를 순서대로 참고하며, 매 시점 요약문을 작성합니다.   
요약문과 새로운 청크를 비교하여, 요약문을 업데이트합니다.

In [16]:
# 청크 순서대로 각 청크들을 요약하고, 앞부분 청크 요약과 현재 청크를 연결하여 요약을 업데이트하며 진행한다.
# 시간은 오래 걸릴 수 있으나, 청크 업데이트가 잦은 경우에 사용
first_summarize_prompt = ChatPromptTemplate.from_messages([
    ('system', '''당신은 인공지능 전문가입니다.
주어진 논문의 내용을 읽고 한국어로 요약하세요.
요약은 1개의 문단과 문단별 5개의 문장으로 작성하세요.
답변은 한국어로 작성하세요.'''),
    ('user', '''{text}''')])



X = first_summarize_prompt.format_messages(text=document_list[0])

intermediate_summary = gemma2.invoke(X).content
print(intermediate_summary)

본 논문은 'Transformer'라는 새로운 신경망 아키텍처를 소개하며, 이 아키텍처가 기존의 순환 신경망(RNN) 기반 모델보다 뛰어난 번역 품질을 달성하면서도 병렬 처리 성능이 뛰어나다는 것을 보여줍니다. Transformer는 주의 메커니즘만을 사용하여 입력과 출력 사이의 글로벌 의존성을 학습하며, 순환 구조 없이 모든 계산을 병렬로 수행할 수 있습니다. 

* 논문은 기존의 순환 신경망 기반 모델의 한계점을 지적하며, 특히 길어지는 입력/출력 시퀀스에서 병렬 처리의 어려움을 언급합니다.
* Transformer는 자기 주의 메커니즘을 기반으로 하며, 입력 시퀀스 내 모든 위치 간의 관계를 효과적으로 학습할 수 있습니다.
* Transformer는 인코더와 디코더 스택으로 구성되며, 각 스택은 여러 개의 자기 주의 및 점별 밀집 연결 계층으로 이루어져 있습니다.
* 디코더는 인코더의 출력에 대한 주의 메커니즘을 추가하여 이전에 생성된 출력을 활용하여 다음 출력을 생성합니다.
* 논문은 Transformer가 기존 모델보다 우수한 번역 성능을 달성하면서도 훨씬 빠른 학습 속도를 보여주는 실험 결과를 제시합니다.





In [17]:
# Refine Prompt

refine_prompt = ChatPromptTemplate.from_messages([
    ('system', '''당신은 인공지능 전문가입니다.
논문의 현재 시점까지의 한국어 요약이 주어집니다.
이를 읽고, 새롭게 주어지는 내용과 비교하여 한국어 요약을 보완하거나 수정하세요.
전체 요약은 10문장 이내로 작성하세요.
'''),
    ('user', '''현재 시점까지의 요약: {previous_summary}
---
새로운 내용: {new_text}''')])


refine_chain = refine_prompt | gemma2 | StrOutputParser()

for i in tqdm(range(1, len(document_list))):
    intermediate_summary = refine_chain.invoke(
        {'previous_summary':intermediate_summary,
         'new_text':document_list[i].page_content})
    print('')
    print(intermediate_summary)
    print('=======================')
# 길이를 지정하지 않으면 오래 걸릴 수 있습니다.



 25%|██▌       | 1/4 [00:36<01:49, 36.57s/it]


본 논문은 'Transformer'라는 새로운 신경망 아키텍처를 소개하며, 이 아키텍처가 기존의 순환 신경망(RNN) 기반 모델보다 뛰어난 번역 품질을 달성하면서도 병렬 처리 성능이 뛰어나다는 것을 보여줍니다. Transformer는 주의 메커니즘만을 사용하여 입력과 출력 사이의 글로벌 의존성을 학습하며, 순환 구조 없이 모든 계산을 병렬로 수행할 수 있습니다. 

Transformer는 **Multi-Head Attention** 이라는 새로운 메커니즘을 사용합니다. 이는 여러 개의 독립적인 "attention head"를 병렬로 실행하여 입력 시퀀스 내 모든 위치 간의 관계를 효과적으로 학습합니다. 각 head는  다른 가중치를 가지며, 다양한 정보를 학습하여 전체적인 이해력을 향상시킵니다. 

또한, Transformer는 **Positional Encoding**을 사용하여 순서 정보를 모델에 직접적으로 주입합니다. 이는 순환 구조나 합성곱 구조가 없는 Transformer가 시퀀스의 순서를 이해하기 위해 필요한 정보를 제공합니다. 

논문은 Transformer가 기존 모델보다 우수한 번역 성능을 달성하면서 훨씬 빠른 학습 속도를 보여주는 실험 결과를 제시합니다. 특히, Transformer는 RNN 기반 모델보다 훨씬 더 긴 시퀀스를 효율적으로 처리할 수 있습니다. 





 50%|█████     | 2/4 [01:18<01:19, 39.59s/it]


본 논문은 'Transformer'라는 새로운 신경망 아키텍처를 소개하며, 이 아키텍처가 기존의 순환 신경망(RNN) 기반 모델보다 뛰어난 번역 품질을 달성하면서도 병렬 처리 성능이 뛰어나다는 것을 보여줍니다. Transformer는 주의 메커니즘만을 사용하여 입력과 출력 사이의 글로벌 의존성을 학습하며, 순환 구조 없이 모든 계산을 병렬로 수행할 수 있습니다. 

Transformer는 **Multi-Head Attention** 이라는 새로운 메커니즘을 사용합니다. 이는 여러 개의 독립적인 "attention head"를 병렬로 실행하여 입력 시퀀스 내 모든 위치 간의 관계를 효과적으로 학습합니다. 각 head는  다른 가중치를 가지며, 다양한 정보를 학습하여 전체적인 이해력을 향상시킵니다. 

또한, Transformer는 **Positional Encoding**을 사용하여 순서 정보를 모델에 직접적으로 주입합니다. 이는 순환 구조나 합성곱 구조가 없는 Transformer가 시퀀스의 순서를 이해하기 위해 필요한 정보를 제공합니다. 

Transformer는 RNN 기반 모델보다 훨씬 더 긴 시퀀스를 효율적으로 처리할 수 있으며, 특히, 자체 주의 메커니즘은 RNN 기반 모델보다 더 빠른 속도로 계산될 수 있습니다.  

논문은 Transformer가 기존 모델보다 우수한 번역 성능을 달성하면서 훨씬 빠른 학습 속도를 보여주는 실험 결과를 제시합니다. 특히, Transformer는 RNN 기반 모델보다 훨씬 더 긴 시퀀스를 효율적으로 처리할 수 있습니다.  

Transformer는 영어 문장 구조 분석과 같은 다른 작업에도 잘 일반화됩니다. 





 50%|█████     | 2/4 [02:01<02:01, 60.96s/it]


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `gemma2-9b-it` in organization `org_01jd3jew9veemay6ep0tb2znnz` service tier `on_demand` on tokens per minute (TPM): Limit 15000, Used 17288, Requested 3013. Please try again in 21.206s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}